In [ ]:
import sys
import mosek

In [ ]:
inf = 0.0

In [ ]:
def streamprinter(text):
    sys.stdout.write(text)
    sys.stdout.flush()

In [ ]:
def main():
    with mosek.Task() as task:
        task.set_Stream(mosek.streamtype.log, streamprinter);
        bkc = [mosek.boundkey.fx, mosek.boundkey.lo, mosek.boundkey.up];
        blc = [30.0,15.0,-inf];
        buc = [30.0,+inf, 25.0];
        bkx = [mosek.boundkey.lo, mosek.boundkey.ra, mosek.boundkey.lo,mosek.boundkey.lo];
        blx = [0.0,0.0, 0.0, 0.0];
        bux = [+inf,10.0, +inf, +inf];
        
        c = [3.0,1.0, 5.0, 1.0];
        asub = [[0, 1], [0, 1, 2], [0, 1], [1, 2]];
        aval = [[3.0, 2.0],[1.0, 1.0, 2.0],[2.0, 3.0],[1.0, 3.0]];
        
        numvar = len(bkx);
        numcon = len(bkc);
        
        task.appendcons(numcon);
        task.appendvars(numvar);
        
        for j in range(numvar):
            task.putcj(j, c[j]);
            task.putvarbound(j, bkx[j], blx[j], bux[j]);
            
            task.putacol(j,asub[j],aval[j]);
            
        for i in range(numcon):
            task.putconbound(i, bkc[i], blc[i], buc[i]);
        
        task.putobjsense(mosek.objsense.maximize);
            
        task.optimize();
        
        task.solutionsummary(mosek.streamtype.msg);
            
        solsta = task.getsolsta(mosek.soltype.bas);
        
        if (solsta == mosek.solsta.optimal):
            xx = task.getxx(mosek.soltype.bas);
            print("Optimal solution: ");
            for i in range(numvar):
                print("x[" + str(i) + "]=" + str(xx[i]));
                
        elif (solsta == mosek.solsta.dual_infeas_cer or solsta == mosek.solsta.prim_infeas_cer):
            print("Primal or dual infeasibility certificate found.\n");
        
        elif solsta == mosek.solsta.unknown:
                print("Unknown solution status");
                
        else:
            print("Other solution status")
        

In [ ]:
 try:
         main()
            
except mosek.Error as e:
    print("ERROR: %s" % str(e.errno))
    if e.msg is not None:
        print("\t%s" % e.msg)
        sys.exit(1)
except:
    import traceback
    traceback.print_exc()
    sys.exit(1)

In [ ]:
del main

In [ ]:
#SDP
import sys
import mosek

In [ ]:
inf = 0.0

In [ ]:
def streamprinter(text):
    sys.stdout.write(text)
    sys.stdout.flush()

In [ ]:
def main():
 # Create atask object and attachlog stream printer
    with mosek.Task()as task:
        task.set_Stream(mosek.streamtype.log, streamprinter);
        
        bkc = [mosek.boundkey.fx,mosek.boundkey.fx]
        blc = [1.0,0.5];
        buc = [1.0,0.5];  #约束方程的bounds
        
        asub = [[0],[1, 2]]
        aval = [[1.0],[1.0, 1.0]] #非矩阵变量的系数
        
        barci = [0,1, 1, 2, 2]
        barcj = [0,0, 1, 1, 2]
        barcval = [2.0, 1.0, 2.0, 1.0, 2.0] #目标函数的系数矩阵
        
        barai = [[0,1, 2],[0, 1, 2,1, 2, 2]];
        baraj = [[0,1, 2],[0, 0, 0,1, 1, 2]];
        baraval = [[1.0, 1.0, 1.0],[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]; 
        #矩阵变量的约束方程的系数矩阵
        
        numvar = 3
        numcon = len(bkc);
        BARVARDIM = [3]
        
        task.appendvars(numvar)
        task.appendcons(numcon)
        
        task.appendbarvars(BARVARDIM)
        task.putcj(0, 1.0)
        
        for j in range(numvar):
             # Set the bounds on variable j
             # blx[j] <= x_j <= bux[j]
            task.putvarbound(j, mosek.boundkey.fr,-inf, +inf)
            
        for i in range(numcon):
            task.putconbound(i, bkc[i], blc[i], buc[i]);
            task.putarow(i, asub[i],  aval[i])

        task.appendafes(3);
        task.putafefentrylist(range(3), range(3), [1.0]*3)
        task.appendaccseq(task.appendquadraticconedomain(3), 0, None)
        
        symc = task.appendsparsesymmat(BARVARDIM[0],barci,barcj,barcval)
        syma0 = task.appendsparsesymmat(BARVARDIM[0],barai[0],baraj[0],baraval[0]);
        
        syma1 = task.appendsparsesymmat(BARVARDIM[0],barai[1],baraj[1],baraval[1])
        task.putbarcj(0, [symc], [1.0])
        task.putbaraij(0,0, [syma0], [1.0])
        task.putbaraij(1,0, [syma1], [1.0])
        
        task.putobjsense(mosek.objsense.minimize)
        task.optimize()
        task.solutionsummary(mosek.streamtype.msg)
        
        prosta = task.getprosta(mosek.soltype.itr)
        solsta = task.getsolsta(mosek.soltype.itr)
        if (solsta == mosek.solsta.optimal):
            xx = task.getxx(mosek.soltype.itr)
            barx = task.getbarxj(mosek.soltype.itr, 0)
            print("Optimal solution:\nx=%s\nbarx=%s" % (xx,barx))
        elif (solsta== mosek.solsta.dual_infeas_cer or
 solsta ==mosek.solsta.prim_infeas_cer):
            print("Primal ordual infeasibility certificate found.\n")
        elif solsta== mosek.solsta.unknown:
            print("Unknown solution status")
        else:
            print("Other solution status")

In [ ]:
try:
    main()
except mosek.MosekException as e:
    print("ERROR: %s"% str(e.errno))
    if e.msg is not None:
        print("\t%s" %e.msg)
        sys.exit(1)
except:
    import traceback
    traceback.print_exc()
    sys.exit(1)

In [ ]:
del main

In [ ]:
def main():
 # Create atask object and attachlog stream printer
    with mosek.Task()as task:
        task.set_Stream(mosek.streamtype.log, streamprinter);
        
        bkc = [mosek.boundkey.fx,mosek.boundkey.fx]
        blc = [1.0,0.5];
        buc = [1.0,0.5];  #约束方程的bounds
        
        
        barci = [0,1, 1, 2, 2]
        barcj = [0,0, 1, 1, 2]
        barcval = [2.0, 1.0, 2.0, 1.0, 2.0] #目标函数的系数矩阵
        
        barai = [[0,1, 2],[0, 1, 2,1, 2, 2]];
        baraj = [[0,1, 2],[0, 0, 0,1, 1, 2]];
        baraval = [[1.0, 1.0, 1.0],[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]; 
        #矩阵变量的约束方程的系数矩阵
        
        numcon = len(bkc);
        BARVARDIM = [3]
        
        task.appendcons(numcon)
        
        task.appendbarvars(BARVARDIM)
        
            
        for i in range(numcon):
            task.putconbound(i, bkc[i], blc[i], buc[i]);

        
        symc = task.appendsparsesymmat(BARVARDIM[0],barci,barcj,barcval)
        syma0 = task.appendsparsesymmat(BARVARDIM[0],barai[0],baraj[0],baraval[0]);
        
        syma1 = task.appendsparsesymmat(BARVARDIM[0],barai[1],baraj[1],baraval[1])
        task.putbarcj(0, [symc], [1.0])
        task.putbaraij(0,0, [syma0], [1.0])
        task.putbaraij(1,0, [syma1], [1.0])
        
        task.putobjsense(mosek.objsense.minimize)
        task.optimize()
        task.solutionsummary(mosek.streamtype.msg)
        
        prosta = task.getprosta(mosek.soltype.itr)
        solsta = task.getsolsta(mosek.soltype.itr)
        if (solsta == mosek.solsta.optimal):
            xx = task.getxx(mosek.soltype.itr)
            barx = task.getbarxj(mosek.soltype.itr, 0)
            print("Optimal solution:\nx=%s\nbarx=%s" % (xx,barx))
        elif (solsta== mosek.solsta.dual_infeas_cer or
 solsta ==mosek.solsta.prim_infeas_cer):
            print("Primal ordual infeasibility certificate found.\n")
        elif solsta== mosek.solsta.unknown:
            print("Unknown solution status")
        else:
            print("Other solution status")

In [ ]:
try:
    main()
except mosek.MosekException as e:
    print("ERROR: %s"% str(e.errno))
    if e.msg is not None:
        print("\t%s" %e.msg)
        sys.exit(1)
except:
    import traceback
    traceback.print_exc()
    sys.exit(1)

In [ ]:
barx

In [ ]:
try:
    main()

In [ ]:
del main

In [ ]:
def main():
    with mosek.Task() as task:
        task.set_Stream(mosek.streamtype.log, streamprinter)
        barvardims =[3, 4]
        task.appendbarvars(barvardims)
        
        task.putbarcblocktriplet([0]*len(C1_v) +[1]*len(C2_v), C1_k + C2_k,
                                 C1_l + C2_l,C1_v + C2_v,)

In [ ]:
main()

In [ ]:
[0]*3 + [1]*3

In [ ]:
with mosek.Task()as task:
        task.set_Stream(mosek.streamtype.log, streamprinter);
 # Set loghandler for debugging ootput
        task.set_Stream(streamtype.log, streamprinter)
 # Append twosymmetric variables ofdimension 3, 4
        barvardims =[3, 4]
        task.appendbarvars(barvardims)
 # Semidefinitepart of objective function
        task.putbarcblocktriplet(
 [0]*len(C1_v) +[1]*len(C2_v), # Which SDPvariable (j)
 C1_k + C2_k, # Entries:(k,l)->v
 C1_l + C2_l,
 C1_v + C2_v,)
 # Append twoconstraints
        task.appendcons(2)
 # First constraint (equality)
        task.putbarablocktriplet(
 [0]*(len(A1_v)+len(A2_v)), # Which constraint (i = 0)
 [0]*len(A1_v) +[1]*len(A2_v), # Which SDPvariable (j)
 A1_k + A2_k, # Entries:(k,l)->v
 A1_l + A2_l,
 A1_v + A2_v,
 )
 # Second constraint (X2)_{1,2} <= k
        task.putbarablocktriplet(
 [1], # Which constraint (i = 1)
 [1], # Which SDPvariable (j = 1)
 [1], [0], [0.5] # Entries:(k,l)->v
 )
    
    # Set boundsfor constraints
        task.putconboundlist([0,1], [boundkey.fx, boundkey.up],[b,-inf],[b, k])
 # Write theproblem for human inspection
        task.writedata("test.ptf")
 # Optimize
        task.optimize()
        task.solutionsummary(streamtype.msg)
 # Get statusinformation about thesolution
        solsta = task.getsolsta(soltype.itr)
        if solsta ==solsta.optimal:
 # Assuming the optimization succeeded read solution
            print("Solution (lower-triangular part vectorized): ")
            for i in range(2):
                X = task.getbarxj(soltype.itr, i)
                print("X{i} ={X}".format(i=i, X=X))
        elif (solsta== solsta.dual_infeas_cer or solsta ==solsta.prim_infeas_cer):
            print("Primal ordual infeasibility certificate found.\n")
        elif solsta== solsta.unknown:
            print("Unknown solution status")
        else:
            print("Other solution status")

